# Example of visualising the agent's training history performance

`WEIGHTS` : Path to the pre-trained weights
`METRICS` : Path to the training history, or None

In [ ]:
WEIGHTS = './output/weights-25000.pth'
METRICS = './output/metrics.csv'

In [ ]:
import sys
import torch
import gymnasium as gym
import matplotlib.pyplot as plt

sys.path.append("../")
from DQN import VisionDeepQ                                                                 # noqa

sys.path.append("../../")
from help.visualisation.plot import graph                                                   # noqa
from help.visualisation.gif import gif                                                      # noqa

## Parameters

In [ ]:
network = {
    "input_channels": 4, "outputs": 5,
    "channels": [128, 64],
    "kernels": [2, 2],
    "padding": ["valid", "valid"],
    "strides": [2, 1],
    "nodes": [128],
}
optimizer = {"optimizer": torch.optim.RMSprop, "lr": 0.0025}
skip = 3

## Setup

# TODO

# UPDATE LATEX + README TO CORRESPOND TO TETRIS IMAGE

# MENTION REWARD ALGORITHM 

In [ ]:
value_agent = VisionDeepQ(
    network=network, optimizer=optimizer,
    exploration_rate=0.001,
)

weights = torch.load(WEIGHTS, map_location=torch.device('cpu'))
value_agent.load_state_dict(weights)

environment = gym.make('ALE/Tetris-v5', render_mode="rgb_array",
                       obs_type="grayscale", frameskip=1, repeat_action_probability=0.0)
environment.metadata["render_fps"] = 30

## Visualise

### Plotting the metrics from the csv-file created during training.

In [ ]:
graph(METRICS, title="Tetris (RAM)", window=200) if METRICS else None
plt.savefig('./metrics.png') if METRICS else None
plt.show() if METRICS else None

### Creating and saving a gif of the agent in action. The gif will be saved to the given path.

In [ ]:
gif(environment, value_agent, '../tetris.gif', skip, 40)